In [25]:
import pandas as pd
import numpy as np
import os
import time

In [133]:
os.getcwd()

'C:\\Users\\johannes\\Documents\\investment_projects\\Auto-BAHN\\stock_data'

In [13]:
stock_info = pd.read_excel('companylist.xlsx')

stock_info.head()

,Symbol,Name,MarketCap,IPOyear,Sector,industry
0,PIH,"1347 Property Insurance Holdings, Inc.",$43.55M,2014,Finance,Property-Casualty Insurers
1,TURN,180 Degree Capital Corp.,$52.6M,n/a,Finance,Finance/Investors Services
2,FLWS,"1-800 FLOWERS.COM, Inc.",$610.88M,1999,Consumer Services,Other Specialty Stores
3,FCCY,1st Constitution Bancorp (NJ),$137.99M,n/a,Finance,Savings Institutions
4,SRCE,1st Source Corporation,$1.26B,n/a,Finance,Major Banks


In [24]:
stock_info.shape

(3270, 6)

In [128]:
'''
Looping through the urls to extract the financial statement information for a list of companies. 
'''

sheets = ['income-statement', 'balance-sheet', 'cash-flow', 'ratios']
# tickers = stock_info.Symbol.values

tickers = ['aapl']

from urllib.request import urlopen
from bs4 import BeautifulSoup as soup

data = {}
for ticker in tickers:
    holder = {}
    for sheet in sheets:

        url = 'http://www.nasdaq.com/symbol/{t}/financials?query={s}'.format(s = sheet, t = ticker)
        print(url)
        with urlopen(url) as u:
            site = u.read()
        s = soup(site, 'lxml')

        s1 = s.body.findAll('div', id = 'container')[0].findAll('div', id = 'content')[0].findAll('div',id = 'left-column-div')[0].findAll('div', id = 'financials-iframe-wrap')[0].findAll('div',{ "class" : "genTable" })[0]

        df = []
        table = s1.table

        period = table.tr.findAll('th')
        period = [i.text for i in period]
        del period[1]
        df.append(period)

        labels = [i.text for i in table.findAll('th')][6:]
        for i in range(len(labels)):
            a = [labels[i]] + [i.text for i in table.findAll('tr', recursive = False)[i].findAll('td')[-4:]]
            df.append(a)

        df = pd.DataFrame(df)
        df.columns = df.iloc[0,:]
        df = df.reindex(df.index.drop(0))
        df = df.set_index('Period Ending:')
        df.dropna(inplace = True)
        df = df.apply(lambda x: x.replace('$','').replace(',',''))
                
        holder[sheet] = df
        time.sleep(1)

    data[ticker] =  holder

                  

http://www.nasdaq.com/symbol/aapl/financials?query=income-statement
http://www.nasdaq.com/symbol/aapl/financials?query=balance-sheet
http://www.nasdaq.com/symbol/aapl/financials?query=cash-flow
http://www.nasdaq.com/symbol/aapl/financials?query=ratios


In [129]:
# for key, value in data.items():
#     for k, v in value.items():
#         print(k, v)

income-statement 0                                                  9/24/2016     9/26/2015  \
Period Ending:                                                               
Total Revenue                                   $215,639,000  $233,715,000   
Cost of Revenue                                 $131,376,000  $140,089,000   
Gross Profit                                     $84,263,000   $93,626,000   
Research and Development                         $10,045,000    $8,067,000   
Sales, General and Admin.                        $14,194,000   $14,329,000   
Non-Recurring Items                                       $0            $0   
Other Operating Items                                     $0            $0   
Operating Income                                 $60,024,000   $71,230,000   
Add'l income/expense items                        $1,348,000    $1,285,000   
Earnings Before Interest and Tax                 $61,372,000   $72,515,000   
Interest Expense                               

In [130]:
# 

# header_locations = np.where([df.index.values == 'Current Assets', df.index.values == 'Long-Term Assets', 
#           df.index.values == 'Current Liabilities', df.index.values == 'Stock Holders Equity'])[1]

In [131]:
# header_locations

In [132]:
# df.index